In [1]:
# !pip install torchdata
!pip install torchdata==0.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 72.1 MB/s eta 0:00:00


In [2]:
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 31.4 MB/s eta 0:00:00


In [4]:
!python -m spacy download en_core_web_md

2023-03-21 16:14:15.315454: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-21 16:14:16.377243: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-21 16:14:16.377373: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-21 16:14:16.377396: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [5]:
import torch, torchdata, torchtext
from torch import nn
import torch.nn.functional as F

import random, math, time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#make our work comparable if restarted the kernel
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

cuda


In [6]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [7]:
torch.__version__

'1.13.1+cu116'

In [8]:
torchtext.__version__

'0.14.1'

# 1. Loading Data

In [9]:
# import os
# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'

import datasets

# using "github jupyter code to text" dataset
train = datasets.load_dataset("codeparrot/github-jupyter-code-to-text", split="train")
test = datasets.load_dataset("codeparrot/github-jupyter-code-to-text", split="test")
print(train, test)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/codeparrot___parquet/codeparrot--github-jupyter-code-to-text-cf9b56d996fd17e1/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


Dataset({
    features: ['repo_name', 'path', 'license', 'content'],
    num_rows: 47452
}) Dataset({
    features: ['repo_name', 'path', 'license', 'content'],
    num_rows: 11864
})


# 2. EDA - Simple Investigation

In [10]:
print(train['content'][10])

import pylearn2.utils
import pylearn2.config
import theano
import neukrill_net.dense_dataset
import neukrill_net.utils
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import holoviews as hl
%load_ext holoviews.ipython
import sklearn.metrics

cd ..

settings = neukrill_net.utils.Settings("settings.json")
run_settings = neukrill_net.utils.load_run_settings(
    "run_settings/replicate_8aug.json", settings, force=True)

model = pylearn2.utils.serial.load(run_settings['alt_picklepath'])

c = 'train_objective'
channel = model.monitor.channels[c]

"""
Explanation: The following are the results we've got from online augmentation so far. Some bugs have been fixed by Scott since then so these might be redundant. If they're not redundant then they are very bad.
Loading the pickle
End of explanation
"""


plt.title(c)
plt.plot(channel.example_record,channel.val_record)

c = 'train_y_nll'
channel = model.monitor.channels[c]
plt.title(c)
plt.plot(channel.example_record,channel

In [11]:
print(test['content'][10])

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split as tts

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, auc

from __future__ import division
import warnings
warnings.filterwarnings("ignore")

url = "http://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom.data"
secom = pd.read_table(url, header=None, delim_whitespace=True)

url = "http://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom_labels.data"
y = pd.read_table(url, header=None, usecols=[0], squeeze=True, delim_whitespace=True)

print 'The dataset has {} observations/rows and {} variables/columns.' \
       .format(secom.shape[0], secom.shape[1])
pri

In [ ]:
# train_data.shape

# 3. Data Preprocessing

In [12]:
train_split = [split for text in train['content'] for split in text.split('\n') if split != ""]
test_split = [split for text in test['content'] for split in text.split('\n') if split != ""]

In [13]:
# checking the number of sentences
len(train_split), len(test_split)

(11367363, 2875424)

In [14]:
from spacy.lang.en.stop_words import STOP_WORDS
import spacy
import re

nlp = spacy.load('en_core_web_md')

def preprocessing(sentence):
    
    # removing the html tags and non-English words
    sentence = re.sub("<[^>]*>", "", sentence) 
    sentence = re.sub("[^\x00-\x7F]+", "", sentence) 
    stopwords = list(STOP_WORDS)
    doc = nlp(sentence)
    cleaned_tokens = []
    
    # removing extra spaces or duplicate symbols.......
    for token in doc: 
        if token.text not in stopwords and token.pos_ != 'PUNCT' and token.pos_ != 'SPACE' and \
            token.pos_ != 'SYM' and token.pos_!= 'X':
                cleaned_tokens.append(token.lemma_.lower().strip())
                
    return " ".join(cleaned_tokens)

### Tokenizing

In [15]:
# tokenization
from torchtext.data.utils import get_tokenizer
def yield_tokens(data_iter):
    for text in data_iter: 
        text = preprocessing(text)
        yield tokenizer(text)

tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

train_size = 100000
test_size = 20000
tokenized_dataset_train = yield_tokens(train_split[:train_size])
tokenized_dataset_test = yield_tokens(test_split[:test_size])

In [16]:
print("Sentence: ", train_split[0])
# print("Tokenization: ", tokenized_dataset_train[0])

Sentence:  import numpy as np


In [17]:
# tokenized_dataset_train = yield_tokens(train_split[:train_size])
# tokenized_dataset_test = yield_tokens(test_split[:test_size])

In [18]:
# tokenized_dataset_train[0]

In [19]:
# tokenized_dataset_train

### Numericalizing

In [20]:
# defining special symbols and indices
UNK_IDX, PAD_IDX, SOS_IDX, EOS_IDX = 0, 1, 2, 3
# making sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<sos>', '<eos>']

In [21]:
# numericalization
vocab = torchtext.vocab.build_vocab_from_iterator(yield_tokens(train_split[:train_size]), min_freq=3, specials=special_symbols)
            
vocab.set_default_index(vocab['<unk>'])   
print(len(vocab))                         
print(vocab.get_itos()[:10]) 

11956
['<unk>', '<pad>', '<sos>', '<eos>', '=', 'explanation', '#', 'end', "'", 'import']


# 4. Preparing Data

In [26]:
def get_data(dataset, vocab, batch_size):
    data = []                                                   
    for example in dataset:       
        # appending eos so we know it ends....so model learn how to end...                             
        tokens = example.append('<eos>') # end of sentence
        #numericalization     
        tokens = [vocab[token] for token in example] 
        data.extend(tokens)                                    
    data = torch.LongTensor(data)                                 
    num_batches = data.shape[0] // batch_size # getting the int number of batches...
    data = data[:num_batches * batch_size]    # making the batch evenly, and cut out any remaining                     
    data = data.view(batch_size, num_batches)        
    return data # [batch size, bunch of tokens]
     

In [23]:
# train_data

In [27]:
batch_size = 128
train_data = get_data(tokenized_dataset_train, vocab, batch_size)
valid_data = get_data(tokenized_dataset_test, vocab, batch_size)
# test_data  = get_data(tokenized_dataset['test'], vocab, batch_size)

In [28]:
# train_loader

In [29]:
import pickle

# saving train and valid data
object_data = train_data
file_train = open('train_data.pkl', 'wb') 
pickle.dump(object_data, file_train)

object_data = valid_data
file_val = open('valid_data.pkl', 'wb') 
pickle.dump(object_data, file_val)

# saving vocab
object_data = vocab
file_vocab = open('vocab.pkl', 'wb') 
pickle.dump(object_data, file_vocab)

# 5. Modeling

## Mutli Head Attention Layer

In [56]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        # query = [batch size, query len, hid dim]
        # key = [batch size, key len, hid dim]
        # value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        # Q = [batch size, query len, hid dim]
        # K = [batch size, key len, hid dim]
        # V = [batch size, value len, hid dim]
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        # Q = [batch size, n heads, query len, head dim]
        # K = [batch size, n heads, key len, head dim]
        # V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        # energy = [batch size, n heads, query len, key len]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)
        # attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        # x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        # x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        # x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        # x = [batch size, query len, hid dim]
        
        return x, attention

## Position-wise Feedforward Layer

In [57]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        # x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        # x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        # x = [batch size, seq len, hid dim]
        
        return x

In [58]:
class BeamSearchNode(object):
    def __init__(self, previousNode, wordId, logProb, length):
        self.prevNode = previousNode  # where does it come from
        self.wordid   = wordId  #  numericalized integer of the word
        self.logp     = logProb  # the log probability
        self.len      = length  # the current length; first word starts at 1

    def eval(self, alpha=0.7):
        # the score will be simply the log probability penaltized by the length 
        # we are adding some small number to avoid division error
        # read https://arxiv.org/abs/1808.10006 to understand how alpha is selected
        return self.logp / float(self.len + 1e-6) ** (alpha)
    
    # this is the function for comparing between two beamsearchnodes, whether which one is better
    # it is called when you called "put"
    def __lt__(self, other):
        return self.len < other.len

    def __gt__(self, other):
        return self.len > other.len

## Decoder 

In [59]:
from queue import PriorityQueue
import operator

class Decoder(nn.Module):
    def __init__(self, output_dim, hid_dim, n_layers, n_heads, 
                 pf_dim, dropout, device, pad_idx, max_length = 100):
                
        super().__init__()
        
        self.device = device
        
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
    
        self.pad_idx = pad_idx
    
    def make_mask(self, x):
        
        # x = [batch size, len]
        
        pad_mask = (x != self.pad_idx).unsqueeze(1).unsqueeze(2)
        # pad_mask = [batch size, 1, 1, len]
        
        x_len = x.shape[1]
        
        sub_mask = torch.tril(torch.ones((x_len, x_len), device = self.device)).bool()
        # sub_mask = [len, len]
            
        mask = pad_mask & sub_mask
        # mask = [batch size, 1, len, len]
        
        return mask 
    
    def forward(self, x):
        
        # x = [batch size, len]
                
        batch_size = x.shape[0]
        x_len    = x.shape[1]
        
        # getting mask here since we remove seq2seq class
        mask   = self.make_mask(x)
        # mask = [batch size, 1, len, len]

        pos = torch.arange(0, x_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)          
            
        x = self.dropout((self.tok_embedding(x) * self.scale) + self.pos_embedding(pos))
        # x = [batch size, len, hid dim]
        
        for layer in self.layers:
            x, attention = layer(x, mask)
        
        # x = [batch size, len, hid dim]
        # attention = [batch size, n heads, len, len]
        
        output = self.fc_out(x)
        # output = [batch size, len, output dim]
            
        return output, attention

    def beam_decode(self, src_tensor, method='beam-search'):
        
        # src_tensor = [batch size, src len]
        src_len = src_tensor.shape[1]
        
        # how many parallel searches
        beam_width = 3
        
        # how many sentence do you want to generate
        topk = 1  
        
        # final generated sentence
        decoded_batch = []
                                        
        # starting with the start of the sentence token
        decoder_input = torch.LongTensor([SOS_IDX]).to(device)

        # number of sentences to generate
        endnodes = []  # holding the nodes of EOS, so we can backtrack
        number_required = min((topk + 1), topk - len(endnodes))

        # starting node -  hidden vector, previous node, word id, logp, length
        node = BeamSearchNode(None, decoder_input, 0, 1)
        nodes = PriorityQueue()  # this is a min-heap

        # starting the queue
        nodes.put((-node.eval(), node))  # we need to put - because PriorityQueue is a min-heap
        qsize = 1

        # starting beam search
        while True:
            # giving up when decoding takes too long
            if qsize > 100: break
            
            # print(f"{nodes.queue=}")

            # fetching the best node
            # the score is log p divides by the length scaled by some constants
            score, n       = nodes.get()
            decoder_input  = n.wordid

            # getting all the previous nodes to construct a complete decoder input
            # because Transformer decoder expects the whole sentence
            prevNode = n.prevNode
            while prevNode != None:
                prev_word = torch.LongTensor([prevNode.wordid]).to(device)
                # print(f"{prev_word=}")
                decoder_input = torch.cat((decoder_input, prev_word))
                prevNode = prevNode.prevNode

            inv_idx       = torch.arange(decoder_input.size(0)-1, -1, -1).long()
            decoder_input = decoder_input[inv_idx]

            # wordid is simply the numercalized integer of the word
            current_len    = n.len

            decoder_input  = decoder_input.unsqueeze(0)
            # decoder_input: batch_size, src_len

            if n.wordid.item() == EOS_IDX and n.prevNode != None:
                endnodes.append((score, n))
                # if we reached maximum # of sentences required
                if len(endnodes) >= number_required:
                    break
                else:
                    continue

            # decode for one step using decoder
            # decoder_input = SOS_IDX
            # mask = [1, src len]
            decoder_input = F.pad(decoder_input, pad=(0, src_len), mode='constant', value=PAD_IDX)
            # padding because our decoder expects a whole sentence, not one token by token....

#             print(f"{current_len=}")
#             print(f"{decoder_input=}")

            prediction, _ = self.forward(decoder_input)
            # prediction   = [batch size, src len, output dim]

            prediction = prediction[:, current_len, :] # getting only the next word, but ignoring the padding
            # prediction   = [batch size, output dim]

            # so basically prediction is probabilities across all possible vocab
            # we gonna retrieve k top probabilities (which is defined by beam_width) and their indexes
            # we recall that beam_width defines how many parallel searches we want
            log_prob, indexes = torch.topk(prediction, beam_width)
            # log_prob      = (1, beam width)
            # indexes       = (1, beam width)
            
            # print(f"{log_prob.shape}")
            # print(f"{indexes.shape}")

            nextnodes = []  # the next possible node we can move to

            # we only select beam_width amount of nextnodes
            for top in range(beam_width):
                pred_t = indexes[0, top].reshape(-1)  # reshaping because wordid is assume to be []; see when we define SOS
                log_p  = log_prob[0, top].item()

                # decoder previous node, current node, prob, length
                node = BeamSearchNode(n, pred_t, n.logp + log_p, n.len + 1)
                score = -node.eval()
                nextnodes.append((score, node))

            # putting them into queue
            for i in range(len(nextnodes)):
                score, nn = nextnodes[i]
                nodes.put((score, nn))
                # increasing qsize
            qsize += len(nextnodes) - 1


        ### Once everything is finished, we choose nbest paths and back trace them.

        ## in case it does not finish, we simply get couple of nodes with highest probability
        if len(endnodes) == 0:
            endnodes = [nodes.get() for _ in range(topk)]

        # looking from the end and go back....
        utterances = []
        for score, n in sorted(endnodes, key=operator.itemgetter(0)):
            utterance = []
            utterance.append(n.wordid)
            # back tracing by looking at the previous nodes.....
            while n.prevNode != None:
                n = n.prevNode
                utterance.append(n.wordid)

            utterance = utterance[::-1]  # reversing it....
            utterances.append(utterance) # appending to the list of sentences....

        decoded_batch.append(utterances)

        return decoded_batch  # (batch size, length)

## Decoder Layer

In [60]:
class DecoderLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, pf_dim, dropout, device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)        
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, pf_dim, dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        
        # x = [batch size, len, hid dim]
        # mask = [batch size, 1, len, len]
        
        # multi attention, skip and then norm
        _x, attention = self.self_attention(x, x, x, mask)
        x = self.self_attn_layer_norm(x + self.dropout(_x))
        # x = [batch size, len, hid dim]
        # attention = [batch size, n heads, len, len]
    
        # positionwise feedforward
        _x = self.positionwise_feedforward(x)
        x = self.ff_layer_norm(x + self.dropout(_x))
        # x = [batch size, len, hid dim]
        
        return x, attention

# 6. Training

In [61]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [62]:
vocab_size = len(vocab)
hid_dim    = 256                
dec_layers = 3               
dec_heads  = 8
dec_pf_dim = 512
dec_dropout = 0.1     
lr = 1e-3   
model = Decoder(vocab_size, hid_dim, dec_layers, dec_heads, dec_pf_dim, dec_dropout, device, PAD_IDX).to(device)

In [63]:
# we can print the complexity by the number of parameters
def count_parameters(model):
    params = [p.numel() for p in model.parameters() if p.requires_grad]
    for item in params:
        print(f'{item:>6}')
    print(f'______\n{sum(params):>6}')
    
count_parameters(model)

3060736
 25600
   256
   256
   256
   256
 65536
   256
 65536
   256
 65536
   256
 65536
   256
131072
   512
131072
   256
   256
   256
   256
   256
 65536
   256
 65536
   256
 65536
   256
 65536
   256
131072
   512
131072
   256
   256
   256
   256
   256
 65536
   256
 65536
   256
 65536
   256
 65536
   256
131072
   512
131072
   256
3060736
 11956
______
7740340


In [64]:
import torch.optim as optim

lr = 0.0005

# training hyperparameters
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss() # combining softmax with cross entropy

In [65]:
def get_batch(data, seq_len, idx):
    # data # [batch size, bunch of tokens]
    src    = data[:, idx:idx+seq_len]                   
    target = data[:, idx+1:idx+seq_len+1]  # target simply is ahead of src by 1            
    return src, target

In [66]:
from tqdm import tqdm

def train(model, data, optimizer, criterion, batch_size, seq_len, clip, device):
    
    epoch_loss = 0
    model.train()
    # droping all batches that are not a multiple of seq_len
    # data # [batch size, bunch of tokens]
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]  # we need to add -1 because we start at 0
    num_batches = data.shape[-1]
        
    for idx in tqdm(range(0, num_batches - 1, seq_len), desc='Training: ',leave=False):
        optimizer.zero_grad()
        
        src, target = get_batch(data, seq_len, idx) # src, target: [batch size, seq len]
        src, target = src.to(device), target.to(device)
        batch_size = src.shape[0]
        prediction, _ = model(src)               

        # we need to reshape because criterion expects pred to be 2d and target to be 1d
        prediction = prediction.reshape(batch_size * seq_len, -1)  # prediction: [batch size * seq len, vocab size]  
        target = target.reshape(-1)
        loss = criterion(prediction, target)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item() * seq_len
    return epoch_loss / (num_batches + 1)

In [68]:
def evaluate(model, data, criterion, batch_size, seq_len, device):

    epoch_loss = 0
    model.eval()
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]
    num_batches = data.shape[-1]
    
    decoded_batch_list = []

    with torch.no_grad():
        for idx in range(0, num_batches - 1, seq_len):
            src, target = get_batch(data, seq_len, idx)
            src, target = src.to(device), target.to(device)
            
            # target = [batch size, dec len]

            batch_size= src.shape[0]

            prediction, _ = model(src)
            # prediction = [batch size, dec len, output_dim]
            
            # decoding using beam_search as example (here beam_search is for intference)
            decoded_batch = model.beam_decode(src, method='beam-search')
            # print(decoded_batch)
            
            # len(decoded_batch) = 64
            # len(decoded_batch[0]) = 1 = number of sentence generated, i.e., topk            
            decoded_batch_list.append(decoded_batch)
            
            prediction = prediction.reshape(batch_size * seq_len, -1)
            target = target.reshape(-1)

            loss = criterion(prediction, target)
            epoch_loss += loss.item() * seq_len

            # print(vocab)
            # print(type(vocab))
            
        # this is optional; printing first three samples of the first batch
        print("Printing samples from first decode batch:")

        for sentence_index in decoded_batch_list[0][:3]:
            # decode_text_arr = [vocab.lookup_token(i) for i in sentence_index[0]]
            # decode_text_arr = [vocab.itos[i.item()] for i in sentence_index[0]]
            # decode_sentence = " ".join(decode_text_arr)
            decode_text_arr = [vocab.lookup_token(i) for i in sentence_index[0]]
            decode_sentence = " ".join(decode_text_arr)
            print("pred target : {}".format(decode_sentence))
            
    return epoch_loss / (num_batches + 1)

## Putting everything together

In [69]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [70]:
n_epochs = 5
seq_len  = 10 # <----decoding length
clip    = 0.25

lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=0)

best_valid_loss = float('inf')

for epoch in range(n_epochs):

    start_time = time.time()

    train_loss = train(model, train_data, optimizer, criterion, 
                batch_size, seq_len, clip, device)
    valid_loss = evaluate(model, valid_data, criterion, batch_size, 
                seq_len, device)

    lr_scheduler.step(valid_loss)

    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'best-val-tr_lm.pt')

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Perplexity: {math.exp(train_loss):.3f}')
    print(f'\tValid Perplexity: {math.exp(valid_loss):.3f}')
      
    # lower perplexity is better

Printing samples from first decode batch:
pred target : <sos> <eos>
Epoch: 01 | Time: 0m 9s
	Train Perplexity: 203.200
	Valid Perplexity: 97.628


Printing samples from first decode batch:
pred target : <sos> <eos>
Epoch: 02 | Time: 0m 8s
	Train Perplexity: 106.499
	Valid Perplexity: 85.935


Printing samples from first decode batch:
pred target : <sos> <eos>
Epoch: 03 | Time: 0m 8s
	Train Perplexity: 76.305
	Valid Perplexity: 81.911


Printing samples from first decode batch:
pred target : <sos> <eos>
Epoch: 04 | Time: 0m 8s
	Train Perplexity: 58.504
	Valid Perplexity: 80.930


Printing samples from first decode batch:
pred target : <sos> <eos>
Epoch: 05 | Time: 0m 9s
	Train Perplexity: 47.275
	Valid Perplexity: 81.790


# 7. Real-world inference

In [71]:
def generate(prompt, max_seq_len, temperature, model, tokenizer, vocab, device, seed=None):
    if seed is not None:
        torch.manual_seed(seed)
    model.eval()
    tokens = tokenizer(prompt)
    indices = [vocab[t] for t in tokens]
    batch_size = 1
    with torch.no_grad():
        for i in range(max_seq_len):
            src = torch.LongTensor([indices]).to(device)
            prediction, _ = model(src)
            
            # prediction: [batch size, seq len, vocab size]
            # prediction[:, -1]: [batch size, vocab size] # probability of last vocab
            
            probs = torch.softmax(prediction[:, -1] / temperature, dim=-1)  
            prediction = torch.multinomial(probs, num_samples=1).item()    
            
            while prediction == vocab['<unk>']: #if it is unk, we sample again
                prediction = torch.multinomial(probs, num_samples=1).item()

            if prediction == vocab['<eos>']:    #if it is eos, we stop
                break

            indices.append(prediction) #autoregressive, thus output becomes input
            
            #####################################################################
            # only pure sampling is done....
            # comparing with top-k, top-p, and beam search can be done here
            #####################################################################

    itos = vocab.get_itos()
    tokens = [itos[i] for i in indices]
    return tokens

In [74]:
prompt = 'import numpy '
max_seq_len = 30
seed = 0

# smaller the temperature, more diverse tokens but comes with a tradeoff of less-make-sense sentence

temperatures = [0.5, 0.7, 0.75, 0.8, 1.0]
for temperature in temperatures:
    generation = generate(prompt, max_seq_len, temperature, model, tokenizer, 
                          vocab, device, seed)
    print(str(temperature)+'\n'+' '.join(generation)+'\n')

0.5
import numpy np

0.7
import numpy np

0.75
import numpy np

0.8
import numpy np

1.0
import numpy np



In [87]:
prompt = 'import tensorflow '
max_seq_len = 30
seed = 0

#smaller the temperature, more diverse tokens but comes 
#with a tradeoff of less-make-sense sentence
temperatures = [0.5, 0.7, 0.75, 0.8, 1.0]
for temperature in temperatures:
    generation = generate(prompt, max_seq_len, temperature, model, tokenizer, 
                          vocab, device, seed)
    print(str(temperature)+'\n'+' '.join(generation)+'\n')

0.5
import tensorflow tf

0.7
import tensorflow

0.75
import tensorflow

0.8
import tensorflow

1.0
import tensorflow

